The best way to handle these puzzles is going into part 1 as simple as possible. 
In part 2 we will have to make drastic changes to the code anyways, but part 2 is still unkown.
So any way of being "clever" or trying to write "good" code for part 1 is foolish and 
migh even be contra-productive. As the ancient saysing goes: 

> Pre-mature optimization is the root of all evil

